In [20]:
# ==========================================================
# EfficientNetV2-S | CIFAR-10 | PyTorch | 5 Seeds | 4 Epochs | GPU Supported
# ==========================================================

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import random
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [21]:
# --------------------------
# Transformations & Dataloaders
# --------------------------
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

In [22]:
# --------------------------
# Model Definition (stronger fine-tuning)
# --------------------------
class EfficientNetV2S_Model(nn.Module):
    def __init__(self):
        super(EfficientNetV2S_Model, self).__init__()
        self.model = torchvision.models.efficientnet_v2_s(weights="IMAGENET1K_V1")

        
        for param in self.model.features.parameters():
            param.requires_grad = False
        for name, param in list(self.model.features.named_parameters())[-70:]:
            param.requires_grad = True

        # Modify classifier for 10 classes
        in_features = self.model.classifier[1].in_features
        self.model.classifier[1] = nn.Linear(in_features, 10)

    def forward(self, x):
        return self.model(x)

In [23]:
# --------------------------
# Training
# --------------------------
def train_model(model, trainloader, criterion, optimizer, epochs=4):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{epochs}] - Loss: {running_loss/len(trainloader):.4f}")


In [24]:
# --------------------------
# Evaluation
# --------------------------
def evaluate_model(model, testloader):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='macro', zero_division=0)
    rec = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    return acc, prec, rec, f1

In [25]:
# --------------------------
# Multi-Seed Training
# --------------------------
seeds = [42, 123, 340, 777, 999]
results = []

for seed in seeds:
    print(f"\n=== Running for seed: {seed} (4 epochs) ===")
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    model = EfficientNetV2S_Model().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=2e-4)

    train_model(model, trainloader, criterion, optimizer, epochs=4)
    acc, prec, rec, f1 = evaluate_model(model, testloader)
    results.append([acc, prec, rec, f1])
    print(f"Seed {seed} → Acc: {acc:.4f}, Prec: {prec:.4f}, Rec: {rec:.4f}, F1: {f1:.4f}")


=== Running for seed: 42 (4 epochs) ===
Epoch [1/4] - Loss: 0.5500
Epoch [2/4] - Loss: 0.3271
Epoch [3/4] - Loss: 0.2680
Epoch [4/4] - Loss: 0.2208
Seed 42 → Acc: 0.9187, Prec: 0.9189, Rec: 0.9187, F1: 0.9186

=== Running for seed: 123 (4 epochs) ===
Epoch [1/4] - Loss: 0.5507
Epoch [2/4] - Loss: 0.3344
Epoch [3/4] - Loss: 0.2642
Epoch [4/4] - Loss: 0.2234
Seed 123 → Acc: 0.9234, Prec: 0.9238, Rec: 0.9234, F1: 0.9233

=== Running for seed: 340 (4 epochs) ===
Epoch [1/4] - Loss: 0.5484
Epoch [2/4] - Loss: 0.3287
Epoch [3/4] - Loss: 0.2683
Epoch [4/4] - Loss: 0.2190
Seed 340 → Acc: 0.9208, Prec: 0.9214, Rec: 0.9208, F1: 0.9208

=== Running for seed: 777 (4 epochs) ===
Epoch [1/4] - Loss: 0.5541
Epoch [2/4] - Loss: 0.3284
Epoch [3/4] - Loss: 0.2631
Epoch [4/4] - Loss: 0.2196
Seed 777 → Acc: 0.9190, Prec: 0.9195, Rec: 0.9190, F1: 0.9190

=== Running for seed: 999 (4 epochs) ===
Epoch [1/4] - Loss: 0.5593
Epoch [2/4] - Loss: 0.3266
Epoch [3/4] - Loss: 0.2674
Epoch [4/4] - Loss: 0.2219
Seed

In [26]:
# --------------------------
# Mean ± SD
# --------------------------
results = np.array(results)
metrics = ["Accuracy", "Precision", "Recall", "F1-Score"]

print("\n=== Final Results (Mean ± SD across 5 seeds) ===")
for i, metric in enumerate(metrics):
    mean_val = results[:, i].mean()
    sd_val = results[:, i].std()
    print(f"{metric}: {mean_val:.4f} ± {sd_val:.4f}")


=== Final Results (Mean ± SD across 5 seeds) ===
Accuracy: 0.9208 ± 0.0018
Precision: 0.9212 ± 0.0018
Recall: 0.9208 ± 0.0018
F1-Score: 0.9207 ± 0.0018
